# 데이터 로딩

In [ ]:
from datasets import load_dataset
import pandas as pd
from pathlib import Path

In [ ]:
# text: subject + " " + message를 공백으로 이어 붙여 만든 분류용 본문(제목과 본문을 합친 텍스트).
ds = load_dataset("SetFit/enron_spam")  # splits: 'train', 'test'
print(ds)  # 구조 확인용

In [ ]:
# 저장 폴더 준비
dir_store = Path("data/enron_spam")
dir_store.mkdir(parents=True, exist_ok=True)

In [ ]:
# 각 스플릿을 CSV로 저장
for split in ds.keys():  # 'train', 'test'
    df = ds[split].to_pandas()
    df.to_csv(dir_store / f"enron_spam_{split}.csv", index=False)

In [ ]:
print("Saved to:", list(dir_store.glob("*.csv")))

# 기본 환경 설정

In [ ]:
import os, re, random, math, json
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch

In [ ]:
# 환경 변수
RANDOM_SEED = 42
DATA_DIR = Path("data")
TRAIN_CSV = DATA_DIR / "enron_spam/enron_spam_train.csv"
TEST_CSV  = DATA_DIR / "enron_spam/enron_spam_test.csv"
OUT_DIR = DATA_DIR / "eron_outputs"

In [ ]:
OUT_DIR.mkdir(parents=True, exist_ok=True)

In [ ]:
# 재현 가능성(reproducibility)을 위해 실험마다 동일한 난수를 쓰도록 고정값 설정
torch.manual_seed(RANDOM_SEED) # PyTorch의 CPU 난수 시드(가중치 초기화, dropout 등 torch CPU 연산에서 쓰는 난수에 영향)
torch.cuda.manual_seed_all(RANDOM_SEED) # 모든 GPU(CUDA 디바이스)의 난수 시드(드롭아웃, 일부 커널 내부 샘플링 등 CUDA 연산에 영향)
np.random.seed(RANDOM_SEED) # NumPy 난수 시드(전처리/샘플링에 NumPy를 사용할 때 결과를 고정)
random.seed(RANDOM_SEED) # Python 표준 random 모듈의 시드(데이터 셔플 등 random 모듈 사용 코드의 결과를 고정)
# cuDNN이 비결정적(non-deterministic) 알고리즘을 쓰지 못하게 강제
#  - 장점: 같은 입력과 시드로 항상 같은 출력(결정적 결과)
#  - 단점: 일부 연산이 느려질 수 있음
torch.backends.cudnn.deterministic = True
# 입력 크기별로 가장 빠른 알고리즘을 자동 탐색(benchmark)하는 기능 비활성화
#  - 켜두면 실행마다 선택된 알고리즘이 달라져 결과가 흔들릴 수 있어 재현성 저하
#  - 끄면 속도는 손해 볼 수 있지만 알고리즘 선택이 고정되어 재현성 향상
torch.backends.cudnn.benchmark = False

# EDA

## 데이터 로드

In [ ]:
df_train = pd.read_csv(TRAIN_CSV)
df_test  = pd.read_csv(TEST_CSV)

In [ ]:
df_train.head()

## 결측 제거

In [ ]:
df_train = df_train.dropna(subset=["text","label"]).reset_index(drop=True)
df_test = df_test.dropna(subset=["text","label"]).reset_index(drop=True)

## 추가 Feature 생성

In [ ]:
# 텍스트 길이(문자수) & 라인수
df_train["char_len"] = df_train["text"].str.len()
df_train["line_cnt"] = df_train["text"].str.count("\n") + 1
df_test["char_len"] = df_test["text"].str.len()
df_test["line_cnt"] = df_test["text"].str.count("\n") + 1

In [ ]:
df_train.head()

In [ ]:
# 간단 Subject 추출 (본문 맨 처음 줄이 'Subject:' 패턴인 경우)
def extract_subject(s):
    if not isinstance(s, str):
        return None
    m = re.match(r"(?i)^subject:\s*(.*)", s.splitlines()[0]) if s else None
    return m.group(1).strip() if m else s

In [ ]:
df_train["subject"] = df_train["text"].apply(extract_subject)
df_test["subject"] = df_test["text"].apply(extract_subject)

In [ ]:
df_train.head()

In [ ]:
df_train["label"].value_counts(normalize=True) # 빈도수 대신 비율(각 값의 등장 횟수 ÷ 전체 개수)을 반환

# 길이(문자수) 분포 시각화

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
sns.set_theme(style="whitegrid", context="notebook", rc={
    "axes.spines.top": False,
    "axes.spines.right": False,
})

In [ ]:
def plotCharLengthDistribution(df, col="char_len", up_bound=20_000):
    data = df[col].clip(upper=up_bound)
    plt.figure(figsize=(8, 3))
    ax = sns.histplot(data=data, bins=50, stat="percent")

    median = data.median()
    mean = data.mean()
    q95 = data.quantile(0.95)
    ax.axvline(median, ls="--", lw=1, color="red", label=f"median = {median:.0f}")
    ax.axvline(mean,   ls=":",  lw=1, color="red", label=f"mean = {mean:.0f}")
    ax.axvline(q95,    ls="-.", lw=1, color="red", label=f"quantile 95 = {q95:.0f}")

    ax.set_title("Character length distribution (train)", pad=10)
    ax.set_xlabel(col)
    ax.set_ylabel("percent")
    ax.legend(loc="upper right", frameon=False)

    plt.show()

In [ ]:
plotCharLengthDistribution(df_train, "char_len", 20_000)

In [ ]:
plotCharLengthDistribution(df_test, "char_len")

# 토크나이징 (글자 -> 숫자)

In [ ]:
import sentencepiece as spm
from collections import Counter

In [ ]:
SPM_DIR = OUT_DIR / "tokenizer"
SPM_DIR.mkdir(parents=True, exist_ok=True)
SPM_PREFIX = str(SPM_DIR / "enron_tokenizer")

In [ ]:
sp_input_path = SPM_DIR / "spm_input_train.txt"
df_train["text"].to_csv(sp_input_path, index=False, header=False)

In [ ]:
spm.SentencePieceTrainer.Train(
    input=str(sp_input_path),
    model_prefix=SPM_PREFIX,
    vocab_size=5000,
    model_type="unigram",              # 또는 "bpe"
    character_coverage=0.9995,         # 영문 위주라 0.9995, 멀티언어면 1.0 권장
    user_defined_symbols=["<|PAD|>"],
    # input_sentence_size=200000,        # 대용량일 경우 샘플링
    shuffle_input_sentence=True
)

In [ ]:
tokenizer_sp = spm.SentencePieceProcessor()
tokenizer_sp.load(SPM_PREFIX + ".model")
print("SentencePiece vocab size:", tokenizer_sp.get_piece_size())

In [ ]:
def sp_pieces(text):
    return tokenizer_sp.encode_as_pieces(text) if isinstance(text, str) else []

In [ ]:
df_train["tok_len"] = df_train["text"].apply(lambda x: len(sp_pieces(x)))
df_test["tok_len"]  = df_test["text"].apply(lambda x: len(sp_pieces(x)))

In [ ]:
df_train.tail()

In [ ]:
plotCharLengthDistribution(df_train, "tok_len", 4_000)

In [ ]:
sample_for_freq = df_train.sample(n=min(5000, len(df_train)), random_state=RANDOM_SEED)
cnt = Counter(tok for txt in sample_for_freq["text"] for tok in sp_pieces(txt))
top20 = cnt.most_common(50)
print("Top 20 tokens:", top20)

# 머신러닝 베이스라인 (SentencePiece → TF-IDF → 로지스틱/선형 SVM/나이브베이즈)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
import joblib

In [ ]:
# train을 다시 train/val 분리 (모델 선택용)
train_texts = df_train["text"].tolist()
train_labels = df_train["label"].astype(int).tolist()
X_tr, X_val, y_tr, y_val = train_test_split(
    train_texts, train_labels, test_size=0.1, random_state=RANDOM_SEED, stratify=train_labels
)

In [ ]:
# 공통 TF-IDF (unigram+bigram 권장, 필요시 조정)
# (1,2)나 (1,3)처럼 늘리면 **연속 토큰 패턴(콜로케이션)**을 잡아내 표현력이 증가합니다.
# SentencePiece처럼 서브워드 토큰을 쓸 때는, 바이그램이 실제 “단어” 수준 조합을 어느 정도 복원해 주는 효과가 있어 도움이 되는 경우가 많습니다.
tfidf = TfidfVectorizer(
    tokenizer=sp_pieces,
    lowercase=False,    # SentencePiece는 케이스 정보 유지해도 OK
    ngram_range=(1, 2), # 몇 개의 연속 토큰을 하나의 특징으로 만들지를 정하는 파라미터: uni+bi
    min_df=2,           # document frequency(문서 빈도) 하한선. “최소 몇 개 문서에서 등장한 토큰만 어휘(vocabulary)에 남길 것인가”를 정함.
    max_features=5000,  # 단어(특징) 사전의 최대 크기를 제한. 말뭉치 전체에서 등장 빈도(term frequency)가 높은 순으로 정렬해 상위 max_features개만 어휘에 남깁
    preprocessor=None,
    token_pattern=None,     # custom tokenizer 사용시 None
)

In [ ]:
Xtr = tfidf.fit_transform(X_tr)
Xva = tfidf.transform(X_val)
Xte = tfidf.transform(df_test["text"].tolist())

In [ ]:
Xtr.shape, len(y_tr)

In [ ]:
def printTfidfInfo(idx=0):
    i = 2  # 1번째 샘플 (Python은 0부터 시작)
    row = Xtr[i]            # shape: (1, vocab_size) 인 CSR 행렬

    # 어휘(특징) 이름
    feat_names = np.array(tfidf.get_feature_names_out())

    print("=== Xtr[0] 기본 정보 ===")
    print("type:", type(row))
    print("shape:", row.shape)
    print("nnz(해당 희소 행에서 0이 아닌 원소의 개수):", row.nnz)

    # 이 문서에서 등장한(가중치>0) 특징들과 가중치
    idx = row.indices       # 비영零 항목의 열 인덱스들
    val = row.data          # 해당 TF-IDF 값들

    # 가중치 내림차순 상위 20개 토큰만 보기
    order = np.argsort(-val)
    topk = order[:20]

    print("\n=== Xtr[0] Top-20 특징(토큰/바이그램) by TF-IDF ===")
    for j in topk:
        print(f"{feat_names[idx[j]]:30s} {val[j]:.6f}")

In [ ]:
printTfidfInfo(0)

In [ ]:
# 3-3) 세 가지 분류기 학습 & 검증
models = {
    "logreg": LogisticRegression(max_iter=2000, n_jobs=1),
    "linear_svm": LinearSVC(),
    "mnb": MultinomialNB()
}

In [ ]:
val_scores = {}
for name, clf in models.items():
    clf.fit(Xtr, y_tr)
    pred_val = clf.predict(Xva)
    acc = accuracy_score(y_val, pred_val)
    val_scores[name] = acc
    print(f"[VAL] {name}: acc={acc:.4f}")

In [ ]:
best_name = max(val_scores, key=val_scores.get)
best_model = models[best_name]
print("Best on val:", best_name, val_scores[best_name])

In [ ]:
# 3-4) 테스트 평가
pred_test = best_model.predict(Xte)
print(classification_report(df_test["label"].astype(int).tolist(), pred_test, digits=4))

In [ ]:
# 3-5) 혼동행렬 저장
cm = confusion_matrix(df_test["label"].astype(int).tolist(), pred_test)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=[0, 1], yticklabels=[0, 1])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

# Step 4. 딥러닝 베이스라인 (SentencePiece ID → TextCNN)

## 환경 구성

In [ ]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

In [ ]:
device = "cuda" if torch.cuda.is_available() else ("mps" if getattr(torch.backends, "mps", None) and torch.backends.mps.is_available() else "cpu")
device

In [ ]:
# 4-1) 하이퍼파라미터
VOCAB_SIZE = tokenizer_sp.get_piece_size()
PAD_ID = tokenizer_sp.piece_to_id("<|PAD|>") # 우리가 user_defined_symbols로 추가
UNK_ID = tokenizer_sp.unk_id()
MAX_LEN = 512
BATCH_SIZE = 64
EMBED_DIM = 128
FILTERS = 128
KERNEL_SIZES = [3,4,5]
DROPOUT = 0.2
EPOCHS = 20
LR = 1e-3

## 데이터 전처리

In [ ]:
# 4-2) 토큰화/패딩
def encode_ids(text, max_len=MAX_LEN):
    ids = tokenizer_sp.encode_as_ids(text)[:max_len]
    if len(ids) < max_len:
        ids = ids + [PAD_ID] * (max_len - len(ids))
    return np.array(ids, dtype=np.int64)

In [ ]:
class EnronDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, i):
        x = encode_ids(self.texts[i])
        y = int(self.labels[i])
        return torch.tensor(x), torch.tensor(y)

In [ ]:
# 4-3) split (train/val은 앞서 만든 걸 재사용)
train_ds = EnronDataset(X_tr, y_tr)
val_ds   = EnronDataset(X_val, y_val)
test_ds  = EnronDataset(df_test["text"].tolist(), df_test["label"].astype(int).tolist())

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)
test_loader  = DataLoader(test_ds,  batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

In [ ]:
xb, yb = next(iter(train_loader))

In [ ]:
xb.shape, yb.shape

## 모델 생성

In [ ]:
# TextCNN: 여러 크기의 1D 콘볼루션으로 n-그램 특징을 뽑고,
# 각 필터의 "시간 차원"에 대해 글로벌 맥스 풀링 후 모두 이어 붙여 분류하는 모델
class TextCNN(nn.Module):
    def __init__(
        self,
        vocab_size,            # 전체 단어 수 (임베딩 테이블의 행 수)
        embed_dim,             # 임베딩 차원 수 (E)
        num_classes=2,         # 분류할 클래스 개수
        kernel_sizes=[3, 4, 5],# 사용할 1D 커널(필터) 길이들: 3-그램, 4-그램, 5-그램
        num_filters=128,       # 각 커널 크기마다 뽑을 채널(필터) 수 (C)
        dropout=0.5,           # 드롭아웃 비율
        pad_idx=0              # 패딩 토큰의 인덱스 (이 행은 학습 중 업데이트되지 않음)
    ):
        super().__init__()
        # (vocab_size, embed_dim) 크기의 임베딩 테이블. 입력 x는 torch.long이어야 함.
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=pad_idx)

        # Conv1d(in_channels=E, out_channels=C, kernel_size=k)
        # Conv1d는 입력을 (B, 채널, 길이)로 받기 때문에 forward에서 transpose로 바꿉니다.
        self.convs = nn.ModuleList([
            nn.Conv1d(in_channels=embed_dim, out_channels=num_filters, kernel_size=k)
            for k in kernel_sizes
        ])

        self.dropout = nn.Dropout(dropout)

        # 최종 특징 벡터 크기: (C * 커널크기개수). 여기에 선형층으로 클래스 로짓 출력.
        self.fc = nn.Linear(num_filters * len(kernel_sizes), num_classes)

    def forward(self, x):
        # x: (B, L)  — 배치 크기 B, 문장 길이 L (각 토큰은 정수 인덱스)
        emb = self.embedding(x)                # (B, L, E) — 토큰을 임베딩 벡터로 변환
        emb = emb.transpose(1, 2)              # (B, E, L) — Conv1d가 기대하는 형태(채널=E)

        # 각 커널 크기별로: Conv1d -> ReLU
        # conv(emb): (B, C, L-k+1)
        conv_outs = [torch.relu(conv(emb)) for conv in self.convs]

        # "시간(시퀀스) 차원"에 대해 글로벌 맥스 풀링:
        # dim=2가 길이 축이므로 결과는 (B, C)
        pooled = [torch.max(co, dim=2).values for co in conv_outs]  # .values만 사용

        # 여기서 커널별 (B, C) 특징들을 피처 차원(dim=1)으로 이어붙여 (B, C * len(K))로 만듭니다.
        cat = torch.cat(pooled, dim=1)         # 여러 텐서를 지정한 축으로 이어붙여(concatenate) 하나의 텐서로 만듬
                                               # 예: 커널 3개라면 (B, 3*C)

        cat = self.dropout(cat)                # (B, C * len(K))에 드롭아웃

        return self.fc(cat)                    # (B, num_classes) — 클래스별 로짓


In [ ]:
model = TextCNN(
    VOCAB_SIZE, EMBED_DIM,
    num_classes=2, kernel_sizes=KERNEL_SIZES, num_filters=FILTERS, dropout=DROPOUT, pad_idx=PAD_ID
).to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=LR)

In [ ]:
model

## 학습

In [ ]:
from sklearn.metrics import f1_score, accuracy_score

In [ ]:
def evaluate(loader):
    model.eval()
    all_y, all_p = [], []
    with torch.no_grad():
        for xb, yb in loader:
            xb, yb = xb.to(device), yb.to(device)
            logits = model(xb)
            preds = logits.argmax(dim=1)
            all_y.extend(yb.cpu().numpy().tolist())
            all_p.extend(preds.cpu().numpy().tolist())
    acc = accuracy_score(all_y, all_p)
    f1  = f1_score(all_y, all_p)
    return acc, f1

In [ ]:
best_val = -1
for epoch in range(1, EPOCHS+1):
    model.train()
    total_loss = 0.0
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        logits = model(xb)
        loss = criterion(logits, yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * xb.size(0)
    train_loss = total_loss / len(train_loader.dataset)
    val_acc, val_f1 = evaluate(val_loader)
    print(f"Epoch {epoch}: train_loss={train_loss:.4f}  val_acc={val_acc:.4f}  val_f1={val_f1:.4f}")
    # 간단한 early-best 저장
    if val_f1 > best_val:
        best_val = val_f1
        torch.save(model.state_dict(), OUT_DIR/"textcnn_best.pt")
        print("  (saved best)")

In [ ]:
# 4-6) 테스트 평가
model.load_state_dict(torch.load(OUT_DIR/"textcnn_best.pt", map_location=device))
test_acc, test_f1 = evaluate(test_loader)
print(f"[TEST] TextCNN acc={test_acc:.4f}  f1={test_f1:.4f}")